In [1]:
import json
import pandas as pd

In [2]:
METHODS = [
    ('Panoptic Deeplab', 'ResNet-50', '../results_panoptic/panoptic_deeplab_rn50_s32.json'),
    # ('Panoptic FPN', 'ResNet-101', '../results_panoptic/panoptic_fpn_r101_fpn_2x_LaRS.json'),
    ('Panoptic FPN', 'ResNet-50', '../results_panoptic/panoptic_fpn_r50_fpn_10x_LaRS.json'),
    ('Panoptic FPN', 'ResNet-101', '../results_panoptic/panoptic_fpn_r101_fpn_10x_LaRS.json'),
    ('MaX-DeepLab', 'ResNet-50*', '../results_panoptic/max_deeplab_s_1025_50k.json'),
    ('Mask2Former', 'ResNet-50', '../results_panoptic/mask2former_r50_lsj_8x2_10x_LaRS.json'),
    ('Mask2Former', 'ResNet-101', '../results_panoptic/mask2former_r101_lsj_8x2_10x_LaRS.json'),
    ('Mask2Former', 'Swin-T', '../results_panoptic/mask2former_swin-t-p4-w7-224_lsj_8x2_10x_LaRS.json'),
    # ('Mask2Former', 'Swin-B', '../results_panoptic/mask2former_swin-b-p4-w12-384-in21k_lsj_8x2_10x_LaRS.json'),
    ('Mask2Former', 'Swin-B', '../results_panoptic/mask2former_swin-b-p4-w12-384-in21k_lsj_8x2_20x_LaRS.json'),
]

In [3]:
data = []
for method_name, backbone, results_path in METHODS:
    with open(results_path, 'r') as file:
        results = json.load(file)
    results['method'] = method_name
    results['backbone'] = backbone
    data.append(results)

df = pd.DataFrame(data).set_index(['method', 'backbone'])

In [4]:
normal_cols = [col for col in df.columns if not col.startswith('a')]
agnostic_cols = [col for col in df.columns if col.startswith('a')]

In [5]:
# Normal PQ metrics (obstacle types)
df.round(2)[normal_cols]

PQ     SQ     RQ  PQ_th  SQ_th  RQ_th  PQ_st  \
method           backbone                                                      
Panoptic Deeplab ResNet-50   34.55  69.91  39.86  13.21  60.50  18.72  91.47   
Panoptic FPN     ResNet-50   39.11  73.84  45.41  20.50  66.57  26.77  88.75   
                 ResNet-101  38.32  73.46  44.42  19.28  66.03  25.28  89.11   
MaX-DeepLab      ResNet-50*  30.47  70.59  34.07   7.76  61.55  10.81  91.04   
Mask2Former      ResNet-50   36.03  71.66  41.33  14.73  62.87  20.24  92.82   
                 ResNet-101  36.66  72.50  41.88  15.64  63.89  21.18  92.72   
                 Swin-T      38.23  72.67  43.80  17.45  64.10  23.48  93.65   
                 Swin-B      39.52  72.86  45.34  18.88  64.12  25.48  94.58   

                             SQ_st  RQ_st  
method           backbone                  
Panoptic Deeplab ResNet-50   95.01  96.22  
Panoptic FPN     ResNet-50   93.22  95.13  
                 ResNet-101  93.26  95.47  
MaX-DeepLab      ResNet-50*  94.67  96.11  
Mask2Former      ResNet-50   95.11  97.56  
                 ResNet-101  95.46  97.08  
                 Swin-T      95.53  98.01  
                 Swin-B      96.19  98.30

In [6]:
df[normal_cols].sort_values('PQ').round(1)

PQ    SQ    RQ  PQ_th  SQ_th  RQ_th  PQ_st  \
method           backbone                                                   
MaX-DeepLab      ResNet-50*  30.5  70.6  34.1    7.8   61.6   10.8   91.0   
Panoptic Deeplab ResNet-50   34.6  69.9  39.9   13.2   60.5   18.7   91.5   
Mask2Former      ResNet-50   36.0  71.7  41.3   14.7   62.9   20.2   92.8   
                 ResNet-101  36.7  72.5  41.9   15.6   63.9   21.2   92.7   
                 Swin-T      38.2  72.7  43.8   17.4   64.1   23.5   93.7   
Panoptic FPN     ResNet-101  38.3  73.5  44.4   19.3   66.0   25.3   89.1   
                 ResNet-50   39.1  73.8  45.4   20.5   66.6   26.8   88.8   
Mask2Former      Swin-B      39.5  72.9  45.3   18.9   64.1   25.5   94.6   

                             SQ_st  RQ_st  
method           backbone                  
MaX-DeepLab      ResNet-50*   94.7   96.1  
Panoptic Deeplab ResNet-50    95.0   96.2  
Mask2Former      ResNet-50    95.1   97.6  
                 ResNet-101   95.5   97.1  
                 Swin-T       95.5   98.0  
Panoptic FPN     ResNet-101   93.3   95.5  
                 ResNet-50    93.2   95.1  
Mask2Former      Swin-B       96.2   98.3

In [15]:
sorted(df['SQ'].round(0).unique(), reverse=True)

[74.0, 73.0, 72.0, 71.0, 70.0]

In [22]:
def format_extreme_values(data, format_string="%.1f"):
    # Get top 3 vals
    data_r = data.round(1)
    vals = sorted(data_r.unique(), reverse=True)
    v1,v2,v3 = vals[:3]
    
    def _fn(x):
        if x == v1:
            text = "\\bm1{%s}" % format_string % x
        elif x == v2:
            text = "\\bm2{%s}" % format_string % x
        elif x == v3:
            text = "\\bm3{%s}" % format_string % x
        else:
            text = format_string % x

        return text
    
    return data_r.apply(_fn)

In [30]:
dfp = df.copy()
for col in dfp.columns.get_level_values(0).unique():
    dfp[col] = format_extreme_values(dfp[col])
dfp

PQ          SQ          RQ       PQ_th  \
method           backbone                                                     
Panoptic Deeplab ResNet-50         34.6        69.9        39.9        13.2   
Panoptic FPN     ResNet-50   \bm2{39.1}  \bm1{73.8}  \bm1{45.4}  \bm1{20.5}   
                 ResNet-101  \bm3{38.3}  \bm2{73.5}  \bm3{44.4}  \bm2{19.3}   
MaX-DeepLab      ResNet-50*        30.5        70.6        34.1         7.8   
Mask2Former      ResNet-50         36.0        71.7        41.3        14.7   
                 ResNet-101        36.7        72.5        41.9        15.6   
                 Swin-T            38.2        72.7        43.8        17.4   
                 Swin-B      \bm1{39.5}  \bm3{72.9}  \bm2{45.3}  \bm3{18.9}   

                                  SQ_th       RQ_th       PQ_st       SQ_st  \
method           backbone                                                     
Panoptic Deeplab ResNet-50         60.5        18.7        91.5        95.0   
Panoptic FPN     ResNet-50   \bm1{66.6}  \bm1{26.8}        88.8        93.2   
                 ResNet-101  \bm2{66.0}  \bm3{25.3}        89.1        93.3   
MaX-DeepLab      ResNet-50*        61.6        10.8        91.0        94.7   
Mask2Former      ResNet-50         62.9        20.2  \bm3{92.8}  \bm3{95.1}   
                 ResNet-101        63.9        21.2        92.7  \bm2{95.5}   
                 Swin-T      \bm3{64.1}        23.5  \bm2{93.7}  \bm2{95.5}   
                 Swin-B      \bm3{64.1}  \bm2{25.5}  \bm1{94.6}  \bm1{96.2}   

                                  RQ_st         aPQ         aSQ         aRQ  \
method           backbone                                                     
Panoptic Deeplab ResNet-50         96.2        76.1        89.0  \bm3{82.7}   
Panoptic FPN     ResNet-50         95.1        75.0        89.2        82.2   
                 ResNet-101        95.5        75.2        89.2        82.4   
MaX-DeepLab      ResNet-50*        96.1        72.0        89.4        77.1   
Mask2Former      ResNet-50   \bm3{97.6}        75.8        89.8        81.5   
                 ResNet-101        97.1  \bm3{76.9}  \bm3{90.3}  \bm3{82.7}   
                 Swin-T      \bm2{98.0}  \bm2{78.1}  \bm2{90.5}  \bm2{83.9}   
                 Swin-B      \bm1{98.3}  \bm1{78.8}  \bm1{90.9}  \bm1{84.2}   

                                 aPQ_th      aSQ_th      aRQ_th      aPQ_st  \
method           backbone                                                     
Panoptic Deeplab ResNet-50         29.9        70.8  \bm3{42.3}        91.5   
Panoptic FPN     ResNet-50   \bm1{33.7}  \bm1{77.3}  \bm1{43.6}        88.8   
                 ResNet-101  \bm2{33.4}  \bm2{77.1}  \bm2{43.3}        89.1   
MaX-DeepLab      ResNet-50*        14.8        73.6        20.1        91.0   
Mask2Former      ResNet-50         24.8        74.0        33.5  \bm3{92.8}   
                 ResNet-101        29.5        74.7        39.5        92.7   
                 Swin-T      \bm3{31.4}  \bm3{75.5}        41.5  \bm2{93.7}   
                 Swin-B            31.3        75.1        41.7  \bm1{94.6}   

                                 aSQ_st      aRQ_st  
method           backbone                            
Panoptic Deeplab ResNet-50         95.0        96.2  
Panoptic FPN     ResNet-50         93.2        95.1  
                 ResNet-101        93.3        95.5  
MaX-DeepLab      ResNet-50*        94.7        96.1  
Mask2Former      ResNet-50   \bm3{95.1}  \bm3{97.6}  
                 ResNet-101  \bm2{95.5}        97.1  
                 Swin-T      \bm2{95.5}  \bm2{98.0}  
                 Swin-B      \bm1{96.2}  \bm1{98.3}

In [33]:
# Normal metrics
print(dfp[normal_cols].to_latex(escape=False))

\begin{tabular}{lllllllllll}
\toprule
            &        &          PQ &          SQ &          RQ &       PQ_th &       SQ_th &       RQ_th &       PQ_st &       SQ_st &       RQ_st \\
method & backbone &             &             &             &             &             &             &             &             &             \\
\midrule
Panoptic Deeplab & ResNet-50 &        34.6 &        69.9 &        39.9 &        13.2 &        60.5 &        18.7 &        91.5 &        95.0 &        96.2 \\
Panoptic FPN & ResNet-50 &  \bm2{39.1} &  \bm1{73.8} &  \bm1{45.4} &  \bm1{20.5} &  \bm1{66.6} &  \bm1{26.8} &        88.8 &        93.2 &        95.1 \\
            & ResNet-101 &  \bm3{38.3} &  \bm2{73.5} &  \bm3{44.4} &  \bm2{19.3} &  \bm2{66.0} &  \bm3{25.3} &        89.1 &        93.3 &        95.5 \\
MaX-DeepLab & ResNet-50* &        30.5 &        70.6 &        34.1 &         7.8 &        61.6 &        10.8 &        91.0 &        94.7 &        96.1 \\
Mask2Former & ResNet-50 &        36.

/tmp/ipykernel_47628/914257788.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dfp[normal_cols].to_latex(escape=False))


In [36]:
# Agnostic PQ metrics (obstacle types ignored)
df[agnostic_cols].reset_index()

,method,backbone,aPQ,aSQ,aRQ,aPQ_th,aSQ_th,aRQ_th,aPQ_st,aSQ_st,aRQ_st
0,Panoptic Deeplab,ResNet-50,76.090289,88.961495,82.734035,29.937890,70.818728,42.273973,91.474422,95.009084,96.220722
1,Panoptic FPN,ResNet-50,74.989039,89.235064,82.248019,33.705754,77.278077,43.616191,88.750134,93.220727,95.125294
2,Panoptic FPN,ResNet-101,75.174460,89.216765,82.426392,33.372329,77.084808,43.293004,89.108503,93.260751,95.470855
3,MaX-DeepLab,ResNet-50*,71.980947,89.407563,77.107230,14.806393,73.616297,20.112928,91.039131,94.671319,96.105330
4,Mask2Former,ResNet-50,75.808566,89.835229,81.543811,24.788814,74.021247,33.488781,92.815151,95.106557,97.562155
5,Mask2Former,ResNet-101,76.916415,90.278516,82.684543,29.514178,74.727119,39.495940,92.717161,95.462315,97.080744
6,Mask2Former,Swin-T,78.082169,90.523448,83.892334,31.370532,75.505839,41.547161,93.652715,95.529318,98.007392
7,Mask2Former,Swin-B,78.758067,90.905624,84.152188,31.303405,75.057451,41.705926,94.576288,96.188349,98.300942


In [43]:
print(dfp[agnostic_cols].reset_index().to_latex(index=False, escape=False))

\begin{tabular}{lllllllllll}
\toprule
          method &   backbone &        aPQ &        aSQ &        aRQ &     aPQ_th &     aSQ_th &     aRQ_th &     aPQ_st &     aSQ_st &     aRQ_st \\
\midrule
Panoptic Deeplab &  ResNet-50 &       76.1 &       89.0 & \bm3{82.7} &       29.9 &       70.8 & \bm3{42.3} &       91.5 &       95.0 &       96.2 \\
    Panoptic FPN &  ResNet-50 &       75.0 &       89.2 &       82.2 & \bm1{33.7} & \bm1{77.3} & \bm1{43.6} &       88.8 &       93.2 &       95.1 \\
    Panoptic FPN & ResNet-101 &       75.2 &       89.2 &       82.4 & \bm2{33.4} & \bm2{77.1} & \bm2{43.3} &       89.1 &       93.3 &       95.5 \\
     MaX-DeepLab & ResNet-50* &       72.0 &       89.4 &       77.1 &       14.8 &       73.6 &       20.1 &       91.0 &       94.7 &       96.1 \\
     Mask2Former &  ResNet-50 &       75.8 &       89.8 &       81.5 &       24.8 &       74.0 &       33.5 & \bm3{92.8} & \bm3{95.1} & \bm3{97.6} \\
     Mask2Former & ResNet-101 & \bm3{76.9} & \bm3{90.

/tmp/ipykernel_47628/2938060321.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dfp[agnostic_cols].reset_index().to_latex(index=False, escape=False))
